# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [2]:
# Initialize and constants
import httpx
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI(http_client=httpx.Client(verify=False))

API key looks good so far


In [3]:
links = fetch_website_links("https://edwarddonner.com")
links

c:\Users\MT331AQ\projects\llm_engineering\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'edwarddonner.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-cou

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [4]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [5]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [6]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/11/11/ai-live-event/
https://edwarddonner.com/2025/11/11/ai-live-event/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
htt

c:\Users\MT331AQ\projects\llm_engineering\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'edwarddonner.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [7]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [8]:
select_relevant_links("https://edwarddonner.com")

c:\Users\MT331AQ\projects\llm_engineering\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'edwarddonner.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'links': [{'type': 'company homepage', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'blog page', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'projects page', 'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'projects page', 'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'LinkedIn profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'Twitter profile', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'Facebook profile',
   'url': 'https://www.facebook.com/edward.donner.52'}]}

In [9]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [10]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gpt-5-nano


c:\Users\MT331AQ\projects\llm_engineering\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'edwarddonner.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Found 9 relevant links


{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'project page', 'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'project page', 'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'blog page', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'company page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'linkedin', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'twitter', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'facebook', 'url': 'https://www.facebook.com/edward.donner.52'}]}

In [ ]:
select_relevant_links("https://huggingface.co")

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [11]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [12]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

c:\Users\MT331AQ\projects\llm_engineering\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Selecting relevant links for https://huggingface.co by calling gpt-5-nano


c:\Users\MT331AQ\projects\llm_engineering\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Found 7 relevant links


c:\Users\MT331AQ\projects\llm_engineering\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\MT331AQ\projects\llm_engineering\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\MT331AQ\projects\llm_engineering\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apply.workable.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  war

## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 2M+ models
Trending on
this week
Models
zai-org/GLM-4.7
Updated
4 days ago
•
4.75k
•
966
Qwen/Qwen-Image-Layered
Updated
8 days ago
•
14.2k
•
774
google/functiongemma-270m-it
Updated
8 days ago
•
30.9k
•
627
Tongyi-MAI/Z-Image-Turbo
Updated
18 days ago
•
403k
•
3.46k
Qwen/Qwen-Image-Edit-2511
Updated
4 days ago
•
11.4k
•
416
Browse 2M+ models
Spaces
Running
on
Zero
Featured
568
TRELLIS.2
🏢
568
High-fidelity 3D Generation from images
Running
on
Zero
Featured
284
Qwen Image Layered
🚀
284
Decompose an image into layers and export as PPTX or ZIP
Running
on
CPU Upgrade
247
Omni Image Editor
🖼
247
Image edit, text to image, face swap, image upscale
Running
on
Zero
832
Z Imag

In [20]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""


In [14]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [15]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

c:\Users\MT331AQ\projects\llm_engineering\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Selecting relevant links for https://huggingface.co by calling gpt-5-nano


c:\Users\MT331AQ\projects\llm_engineering\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Found 8 relevant links


c:\Users\MT331AQ\projects\llm_engineering\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\MT331AQ\projects\llm_engineering\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\MT331AQ\projects\llm_engineering\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apply.workable.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  war

'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 2M+ models\nTrending on\nthis week\nModels\nzai-org/GLM-4.7\nUpdated\n4 days ago\n•\n4.75k\n•\n966\nQwen/Qwen-Image-Layered\nUpdated\n8 days ago\n•\n14.2k\n•\n774\ngoogle/functiongemma-270m-it\nUpdated\n8 days ago\n•\n30.9k\n•\n627\nTongyi-MAI/Z-Image-Turbo\nUpdated\n18 days ago\n•\n403k\n•\n3.46k\nQwen/Qwen-Image-Edit-2511\nUpdated\n4 days ago\n•\n11.4k\n•\n417\nBrowse 2M+ models\nSpaces\nRunning\non\nZero\nFeatured\n568\nTRELLIS.2\

In [16]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))
    return result

In [17]:
brochure= create_brochure("HuggingFace", "https://huggingface.co")

c:\Users\MT331AQ\projects\llm_engineering\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Selecting relevant links for https://huggingface.co by calling gpt-5-nano


c:\Users\MT331AQ\projects\llm_engineering\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Found 14 relevant links


c:\Users\MT331AQ\projects\llm_engineering\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\MT331AQ\projects\llm_engineering\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\MT331AQ\projects\llm_engineering\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warning

# Hugging Face Brochure

---

## About Hugging Face

**Hugging Face** is a pioneering AI community and platform dedicated to building the future of machine learning (ML). It serves as a vibrant hub where millions of ML practitioners, developers, and organizations collaborate on models, datasets, and AI applications to accelerate AI innovation globally.

With over **2 million models**, **500,000+ datasets**, and **1 million+ applications**, Hugging Face empowers users to create, discover, and share cutting-edge machine learning solutions across a wide range of AI modalities including text, image, video, audio, and even 3D data.

---

## Platform Highlights

- **Models:** Access and contribute to an extensive repository of open-source ML models powering tasks like natural language processing, computer vision, and generative AI.
- **Datasets:** Browse and use vast collections of datasets spanning multiple domains and modalities, updated frequently.
- **Spaces:** Host and run interactive ML applications and demos easily on the platform with shared community support.
- **Community:** Join a thriving ecosystem to collaborate, share knowledge, and advance your AI skills alongside experts worldwide.
- **Docs:** Comprehensive documentation and tutorials to help users accelerate their ML projects.
- **Enterprise:** Advanced solutions designed for teams and organizations with features such as enterprise-grade security, access control, dedicated support, and compute resources.

---

## Company Culture

Hugging Face is centered on open collaboration, transparency, and innovation in AI. The culture encourages active participation from individuals and organizations alike to contribute to and benefit from open-source machine learning.

- **Collaborative:** The platform’s core philosophy is to foster collaboration through shared resources and open models.
- **Inclusive:** Welcomes ML researchers, engineers, hobbyists, and companies of all sizes.
- **Empowering:** Provides tools and compute resources to help users move faster and build their AI portfolios.
- **Community Driven:** Features like blogs, forums, and public profiles empower users to showcase expertise and network globally.

---

## Customers & Use Cases

Hugging Face caters to a broad spectrum of users including:

- **Individual ML practitioners:** From beginners to experts building personal portfolios or prototypes.
- **Academic researchers:** Sharing and benchmarking ML models and datasets.
- **Startups and enterprises:** Leveraging paid Compute and Enterprise solutions for scalable, secure AI deployments.
- **Developers and data scientists:** Rapidly experimenting with models and datasets across diverse AI modalities.
- **Innovation labs and AI-driven companies:** Utilizing the platform to accelerate product development and AI-driven research.

---

## Plans & Pricing

Hugging Face offers flexible subscription plans for individuals and organizations:

- **PRO Account — $9/month**
  - Enhanced storage and private dataset capabilities
  - Higher inference credit quotas
  - Priority access to GPU resources
  - Ability to publish blog articles and showcase a Pro badge

- **Team Account — $20/user/month**
  - Instant team setup for collaboration
  - Enterprise features including Single Sign-On (SSO) and Security Assertion Markup (SAML)
  - Access to dedicated support and advanced security controls

Enterprise plans are customizable for larger organizations needing dedicated compute infrastructure and security.

---

## Careeers at Hugging Face

Join a team dedicated to shaping the future of AI in an open, fast-moving, and mission-driven environment. Hugging Face looks for passionate engineers, researchers, community builders, and AI enthusiasts who want to contribute to the open AI ecosystem and empower millions worldwide.

---

## Get Involved

- **Explore models, datasets, and apps:** Dive into over 2 million ML models and 500k+ datasets.
- **Collaborate:** Host your projects and connect with the global AI community.
- **Accelerate your ML**: Upgrade to PRO or Team accounts for enhanced capabilities and resources.
- **Join the community:** Share your work, learn from others, and build a professional ML portfolio.

Visit [huggingface.co](https://huggingface.co) and become part of the AI community building the future today!

---

*Hugging Face – Where Machine Learning Collaboration Thrives*

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [18]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [19]:
stream_brochure("HuggingFace", "https://huggingface.co")

c:\Users\MT331AQ\projects\llm_engineering\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Selecting relevant links for https://huggingface.co by calling gpt-5-nano


c:\Users\MT331AQ\projects\llm_engineering\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Found 8 relevant links


c:\Users\MT331AQ\projects\llm_engineering\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\MT331AQ\projects\llm_engineering\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apply.workable.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\MT331AQ\projects\llm_engineering\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  war

# Hugging Face Brochure

---

## About Hugging Face  
**Hugging Face** is the pioneering AI community and collaboration platform where the machine learning ecosystem comes together to build the future of artificial intelligence. By providing open access to over **2 million models**, **500,000+ datasets**, and **1 million+ applications**, Hugging Face empowers data scientists, researchers, developers, and enterprises to create, share, and innovate in AI across multiple modalities including text, image, video, audio, and 3D.

The platform acts as a central hub fostering open and ethical AI development through collaboration and knowledge sharing. Hugging Face hosts the **HF Hub**, a vibrant space where anyone in the ML community—whether a beginner or expert—can explore, experiment, and contribute to open-source machine learning projects.

---

## What We Offer  
- **Models & Datasets:** Access and contribute to millions of cutting-edge ML models and curated datasets covering all AI domains.  
- **Spaces:** Build and deploy interactive AI apps easily within the community.  
- **Enterprise Solutions:** Secure, scalable platforms with advanced features for teams and organizations including dedicated support and access controls.  
- **Compute Resources:** Paid cloud compute options to accelerate model training and inference.  
- **Open-source Tools & Libraries:** Industry-leading libraries that enable faster AI development.

---

## Who We Serve  
- **Machine Learning Engineers & Researchers** looking for a collaborative environment to build, share, and leverage AI models.  
- **AI Enthusiasts & Learners** who want to build portfolios and learn from a broad community of practitioners.  
- **Enterprises & Teams** seeking a robust platform with enterprise-grade security to accelerate AI projects and deployment.  
- **Developers & Innovators** building AI-powered apps and need seamless access to models and datasets.

---

## Company Culture  
Hugging Face is built on the principles of **open collaboration**, **inclusivity**, and **innovation**. The company thrives as a global community where contributions are celebrated, and learning is continuous. With a diverse and talented science team pushing the edge of technology, Hugging Face fosters a culture where ethical AI development and transparency are paramount.

The organization encourages curiosity, creativity, and sharing of knowledge through active community engagement including blogs, forums, and regular updates on the latest AI research and practical applications.

---

## Careers & Opportunities  
Hugging Face is growing rapidly and actively recruits passionate individuals eager to shape the future of AI. Career openings span multiple areas including machine learning research, software engineering, developer advocacy, and enterprise support.  
Working here means joining a vibrant, diverse team at the heart of the AI revolution, with opportunities for professional growth and contributing to impactful open-source projects.

Explore current openings and join a mission-driven company dedicated to democratizing AI technology.

---

## Join the Hugging Face Community  
- Collaborate on cutting-edge machine learning projects.  
- Share your models and datasets with an enthusiastic global audience.  
- Build your portfolio and reputation within the AI industry.  
- Stay at the forefront of AI technology and research.

**Become a part of the AI community building the future at [huggingface.co](https://huggingface.co).**

---

### Brand Colors  
- Yellow: #FFD21E  
- Orange: #FF9D00  
- Gray: #6B7280

---

For more information, visit our website or follow us on GitHub, Twitter, LinkedIn, and Discord to stay updated with the latest news and opportunities.

In [21]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

c:\Users\MT331AQ\projects\llm_engineering\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Selecting relevant links for https://huggingface.co by calling gpt-5-nano


c:\Users\MT331AQ\projects\llm_engineering\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Found 14 relevant links


c:\Users\MT331AQ\projects\llm_engineering\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\MT331AQ\projects\llm_engineering\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\MT331AQ\projects\llm_engineering\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warning

# Hugging Face  
**The AI Community Building the Future – One Model at a Time!**

---

## Who We Are  
Welcome to Hugging Face, the buzzing hive where machine learning enthusiasts, researchers, and developers unite to dream, build, and share the coolest AI tech out there. Think of us as the social club for ML, but instead of awkward small talk, we swap 2 million+ models, datasets, and AI apps faster than you can say "neural network."

---

## Our Playground  
- **Models:** Explore a vast universe of 2 million+ ML models — from image generation wizards to text transformers. Trending models like *GLM-4.7* or *Qwen Image Edit* are updated constantly to keep your AI cutting-edge.
- **Datasets:** 500,000+ datasets at your fingertips. Whether you want to teach your AI to recognize images or analyze drama scripts (yes, really), we have the data goldmine you need.
- **Spaces:** Showcase your AI apps or discover others' creations. From face swapping to high-fidelity 3D generation, Spaces is where innovation gets its spotlight.
- **Community:** A welcoming village of collaborators, tinkerers, and AI nerds where sharing isn’t just caring — it’s essential!

---

## What Makes Us Different?  
- **Collaboration is Our Jam:** Host and contribute unlimited public models and datasets.
- **Explore All Modalities:** Text, image, video, audio, and yes, even mind-blowing 3D.
- **Portfolio Building:** Share your work, build your ML street cred, and maybe even get hired or funded!
- **Free + Paid Solutions:** Enjoy open-source goodness or scale your projects with our paid Compute and Enterprise tiers.

---

## For Businesses and Enterprises  
Calling all organizations ready to power up their AI game:

- Enterprise-grade security with **Single Sign-On** and **audit logs** (keep those sneaky data leaks out).
- Choose your data regions to tick all compliance checkboxes.
- Dedicated support to untie your team's knottiest ML puzzles.
- Flexible contracts starting at $20/user/month or tailored deals on contact.

Pro Tip: Your team will thank you for giving them the easiest, most advanced AI platform around — productivity + happy workplace = 💥

---

## Company Culture — Because AI Should Be Fun Too  
Here at Hugging Face, we believe great tech is built by *people* who love what they do. We foster:

- **Open-minded collaboration** — hacking away together beats solo struggles.
- **Continuous learning** — from newbies to experts, knowledge flows freely.
- **Playful innovation** — who says ML can’t come with a side of memes and humor?
- **Community-first mindset** — users don’t just use us, they *are* us.

Jump on board, and you’re not just coding — you’re joining a movement.

---

## Careers — Join the AI Revolution!  
Hey, future Hugger! We’re always on the lookout for:

- Machine Learning Engineers ready to build pioneering models.
- DevOps wizards who can make our infrastructure purr.
- Product folks who speak human and AI.
- Community dreamers and moderators who keep the magic alive.

Perks? Work with the smartest folks in a vibrant, inclusive culture that celebrates curiosity, diversity, and making AI accessible to all. Also, lots of coffee and occasional robot jokes.

---

## Ready to Hug Your Future?  
Whether you want to build, explore, or just poke around the coolest AI sandbox on the internet —  
**Jump in. Your AI adventure starts here.**  

[Sign up for free](https://huggingface.co)  
or  
[Explore AI Apps and Models](https://huggingface.co/models)

---

*Hugging Face — We put the “fun” in “functional AI.”* 🤗💻🚀

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>